<a href="https://colab.research.google.com/github/velosomn/M5-methods/blob/master/Criptrouxa_Marcelo_V200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle as pk
import os, sys, gc, time, warnings, pickle, psutil, random
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from math import ceil

warnings.filterwarnings('ignore')
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

In [3]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [4]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
## Merging by concat to not lose dtypes
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition"

Mounted at /content/gdrive
 asset_details.csv
 base_mod.pkl
'Cópia de Cópia de criptotroxa (1).ipynb'
'Cópia de Cópia de criptotroxa (2).ipynb'
'Cópia de Cópia de criptotroxa (3).ipynb'
'Cópia de Cópia de criptotroxa.ipynb'
'Cópia de criptotroxa.ipynb'
'Cópia de criptotroxa Marcelo (1).ipynb'
'Cópia de criptotroxa Marcelo (2).ipynb'
'Cópia de criptotroxa Marcelo (3) (1).ipynb'
'Cópia de criptotroxa Marcelo (3).ipynb'
'Cópia de criptotroxa Marcelo (4).ipynb'
'Cópia de criptotroxa Marcelo.ipynb'
 example_sample_submission.csv
 example_test.csv
 mod_lgbm
 mod_lin_reg
 mod_multi_out
 mod_xgb
 supplemental_train.csv.zip
 treinocompactado.zip


In [ ]:
import zipfile
base_treino=0
with zipfile.ZipFile("gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/treinocompactado.zip", 'r') as zip_ref:
  base_treino = pd.read_csv(zip_ref.open('train.csv'))

ParserError: ignored

In [ ]:
base_treino["dia_semana"]=pd.to_datetime(base_treino['timestamp'], unit='s').dt.dayofweek
base_treino['hora'] = pd.to_datetime(base_treino['timestamp'], unit='s').dt.hour

In [ ]:
base_treino.sample(100)

In [ ]:
base_treino.shape

(24236806, 12)

In [ ]:
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(base_treino.memory_usage(index=True).sum())))

     Reduced grid_df:   2.2GiB


# 1. Estruturar uma base por asset

In [ ]:
assets = {x: base_treino[base_treino.Asset_ID == x].drop('Asset_ID', axis=1) for x in base_treino.Asset_ID.unique()}

In [ ]:
assets[0]["Target"].head()

1    -0.014399
9    -0.015875
17   -0.015410
25   -0.012524
33   -0.005940
Name: Target, dtype: float64

In [ ]:
#base_treino = reduce_mem_usage(base_treino)

Exemplo :
    assets = {2:base_treino[base_treino.Asset_ID==2],3:base_treino[base_treino.Asset_ID==3]...}

In [ ]:
assets[0].head()

,timestamp,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
1,1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
9,1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
17,1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
25,1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
33,1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


# 2. Tratamento inicial

## 2.1 Timestamp como index

In [ ]:
for asset in assets:
    assets[asset] = assets[asset].set_index('timestamp').sort_index()

In [ ]:
assets[0].head()

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora
timestamp,,,,,,,,,,
1514764860,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,0,0
1514764920,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,0,0
1514764980,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,0,0
1514765040,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,0,0
1514765100,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,0,0


## 2.2 Tratamento dos missings

In [ ]:
# Função para uso futuro
def preenche_missing():
    return 0

In [ ]:
for asset in assets:
    assets[asset] = assets[asset].reindex(range(assets[asset].index[0], assets[asset].index[-1] + 60, 60), method='pad').replace([np.inf, -np.inf], np.nan)

In [ ]:
assets[0].info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956960 entries, 1514764860 to 1632182400
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Count       1956960 non-null  float64
 1   Open        1956960 non-null  float64
 2   High        1956960 non-null  float64
 3   Low         1956960 non-null  float64
 4   Close       1956960 non-null  float64
 5   Volume      1956960 non-null  float64
 6   VWAP        1956960 non-null  float64
 7   Target      1929248 non-null  float64
 8   dia_semana  1956960 non-null  int64  
 9   hora        1956960 non-null  int64  
dtypes: float64(8), int64(2)
memory usage: 149.3 MB


In [ ]:
for asset in assets:
    print(f'-----{asset}-----')
    print((assets[asset].index[1:] - assets[asset].index[:-1]).value_counts().head())

-----2-----
60    1956959
Name: timestamp, dtype: int64
-----0-----
60    1956959
Name: timestamp, dtype: int64
-----1-----
60    1956959
Name: timestamp, dtype: int64
-----5-----
60    1956959
Name: timestamp, dtype: int64
-----7-----
60    1956959
Name: timestamp, dtype: int64
-----6-----
60    1956959
Name: timestamp, dtype: int64
-----9-----
60    1956959
Name: timestamp, dtype: int64
-----11-----
60    1956959
Name: timestamp, dtype: int64
-----13-----
60    1903823
Name: timestamp, dtype: int64
-----12-----
60    1889287
Name: timestamp, dtype: int64
-----3-----
60    1803769
Name: timestamp, dtype: int64
-----8-----
60    1772153
Name: timestamp, dtype: int64
-----10-----
60    1770279
Name: timestamp, dtype: int64
-----4-----
60    1285046
Name: timestamp, dtype: int64


# 3. Criação de features

## 3.1 Features gerais - vai ter para todos assets

Fazer a construção de features usando funções, pra facilitar a aplicação nos assets

Features já existentes
* timestamp: All timestamps are returned as second Unix timestamps (the number of seconds elapsed since 1970-01-01 00:00:00.000 UTC). Timestamps in this dataset are multiple of 60, indicating minute-by-minute data.
* Asset_ID: The asset ID corresponding to one of the crytocurrencies (e.g. Asset_ID = 1 for Bitcoin). The mapping from Asset_ID to crypto asset is contained in asset_details.csv.
* Count: Total number of trades in the time interval (last minute).
* Open: Opening price of the time interval (in USD).
* High: Highest price reached during time interval (in USD).
* Low: Lowest price reached during time interval (in USD).
* Close: Closing price of the time interval (in USD).
* Volume: Quantity of asset bought or sold, displayed in base currency USD.
* VWAP: The average price of the asset over the time interval, weighted by volume. VWAP is an aggregated form of trade data.
* Target: Residual log-returns for the asset over a 15 minute horizon.

Ideias de features
* Dia da semana
* Feriados Americanos e Chineses
* Volatilidade exponencial - http://ferramentasdoinvestidor.com.br/gestao-de-risco/volatilidade-ewma-no-excel/
* Indice de força relativa
* Médias moveis
* Bandas de bollinger
* MACD
* Agulhada do DIDI - https://www.investimentonabolsa.com/2014/11/saiba-utilizar-agulhada-do-didi.html
* Analise de fourrier - https://journals.sagepub.com/doi/pdf/10.5772/56839
* Nuvens de ichimoku - https://www.investopedia.com/terms/i/ichimoku-cloud.asp
* numero de transações de bitcoin - observar interesse do publico em cripto

In [ ]:
#def media_movel_30(precos):
#    return precos.rolling(30).mean()

def media_movel(precos,intervalo):
    return precos.rolling(intervalo).mean().astype("float64")

In [ ]:
def get_bollinger_bands(dfbol, rate=30):
    sma = np.array(dfbol.rolling(rate).mean())
    std = np.array(dfbol.rolling(rate).std())
    boll_up =  sma + std * 2 # Calculate top band
    boll_down = sma - std * 2 # Calculate bottom band
    boll=np.zeros(dfbol.size)
    preco=np.array(dfbol)
    indexes = list(range(dfbol.size))
    for i in indexes:
        if boll_up[i]<preco[i]:
            boll[i]=1
        elif boll_down[i]>preco[i]:
            boll[i]=-1
        else:
            boll[i]=0
    return boll

In [ ]:
def agulhada_didi(dfagu):
    didi3 = np.array(dfagu.rolling(3).mean())
    didi8 = np.array(dfagu.rolling(8).mean())
    didi20 = np.array(dfagu.rolling(20).mean())
    Didi=np.zeros(dfagu.size)
    indexes = list(range(dfagu.size))
    for i in indexes:
        if ((didi3[i]>didi8[i]) & (didi8[i]>didi20[i])):
            Didi[i]=1
        else:
            Didi[i]=0
    return Didi

In [ ]:
    def MACD(dfmacd):
        exp1 = dfmacd.ewm(span=12, adjust=False).mean()
        exp2 = dfmacd.ewm(span=26, adjust=False).mean()
        diff = exp1-exp2
        diff_np=np.array(diff)
        exp3 = np.array(diff.ewm(span=9, adjust=False).mean())
        MACD=np.zeros(dfmacd.size)
        indexes = list(range(dfmacd.size))
        for i in indexes:
            if (diff_np[i]>=exp3[i]):
                MACD[i]=1
            else:
                MACD[i]=-1        
        return MACD

In [ ]:
def willR (dfw,dfhigh,dflow,bins):
    high = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dfw,dtype=float)]
    low = [np.array(dfhigh.rolling(bins).max(),dtype=float)-np.array(dflow.rolling(bins).min(),dtype=float)]
    #
    willr = np.transpose(np.nan_to_num(np.true_divide(high,low,out=np.zeros_like(high), where=low!=0)*(-100)))
    
    willr_flag=np.zeros(dfw.size)
    indexes = list(range(dfw.size))
    for i in indexes:
        if (willr[i]>-20):
            willr_flag[i]=1
        elif (willr[i]<-80):
            willr_flag[i]=-1
        else:
            willr_flag[i]=0

    return willr,willr_flag

In [ ]:
def log_return(series, periods=1):
    return np.log(series.astype("float64")).diff(periods=periods)

In [ ]:
def upper_shadow_percent(df): return (df['High'] / np.maximum(df['Close'], df['Open'])) -1
def lower_shadow_percent(df): return (np.minimum(df['Close'], df['Open']) / df['Low']) -1

def upper_shadow_15_perc(df): return (df['High'].rolling(window=15, min_periods=1).max() / np.maximum(df['Close'], df['Open']).shift(15)) -1
def lower_shadow_15_perc(df): return (np.minimum(df['Close'], df['Open']).shift(15) / df['Low'].rolling(window=15, min_periods=1).min()) -1

In [ ]:
for base in assets:
    
    #simple moving average
    for i in [7,15,30,60,180]:
      assets[base]['SMA_'+str(i)] = media_movel(assets[base]['Close'],i)
      assets[base]['SMA_'+str(i)].fillna(method='bfill',inplace=True)
    
    #log simple moving average
    for i in [7,14,30,60,180]:
      assets[base]['log_SMA_'+str(i)] = np.log(media_movel(assets[base]['Close'],i))
      assets[base]['log_SMA_'+str(i)].fillna(method='bfill',inplace=True)

    
    #log cumulative moving average
    assets[base]['log_CMA_30'] = np.log(assets[base]['Close'].expanding(30).mean())
    assets[base]['log_CMA_15'] = np.log(assets[base]['Close'].expanding(15).mean())
    assets[base]['log_CMA_30'].fillna(method='bfill',inplace=True)
    assets[base]['log_CMA_15'].fillna(method='bfill',inplace=True)

    
    #log exponential moving average
    assets[base]['log_EWM_30'] = np.log(assets[base]['Close'].ewm(span=30).mean())
    assets[base]['log_EWM_15'] = np.log(assets[base]['Close'].ewm(span=15).mean())
    assets[base]['log_EWM_30'].fillna(method='bfill',inplace=True)
    assets[base]['log_EWM_15'].fillna(method='bfill',inplace=True)



    #Bollinger
    assets[base]['boll_30'] = get_bollinger_bands(assets[base]['Close'],30)
    #DIDI Index
    assets[base]['Didi'] = agulhada_didi(assets[base]['Close'])
    #MACD
    assets[base]['MACD'] = MACD(assets[base]['Close'])
    #Williams R
    assets[base]['willr'],assets[base]['willr_flag'] = willR(assets[base]['Close'],assets[base]['High'],assets[base]['Low'],14)
    #Log do Retorno
    assets[base]["log_ret"]=log_return(assets[base]['Close'])[1:]
    assets[base]['log_ret'].fillna(method='bfill',inplace=True)

    assets[base]["upper_shadow_perc"]=upper_shadow_percent(assets[base])[1:]
    assets[base]['upper_shadow_perc'].fillna(method='bfill',inplace=True)
    assets[base]["lower_shadow_perc"]=lower_shadow_percent(assets[base])[1:]
    assets[base]['lower_shadow_perc'].fillna(method='bfill',inplace=True)

    assets[base]["upper_shadow_15_perc"]=upper_shadow_15_perc(assets[base])[1:]
    assets[base]['upper_shadow_15_perc'].fillna(method='bfill',inplace=True)

    assets[base]["lower_shadow_15_perc"]=lower_shadow_15_perc(assets[base])[1:]
    assets[base]['lower_shadow_15_perc'].fillna(method='bfill',inplace=True)

    assets[base]["Target"].fillna(method='pad',inplace=True)



In [ ]:
for base in assets:
  assets[base]=reduce_mem_usage(assets[base])

Mem. usage decreased to 123.18 Mb (75.7% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 130.64 Mb (74.3% reduction)
Mem. usage decreased to 126.91 Mb (75.0% reduction)
Mem. usage decreased to 123.18 Mb (75.7% reduction)
Mem. usage decreased to 123.46 Mb (75.0% reduction)
Mem. usage decreased to 122.52 Mb (75.0% reduction)
Mem. usage decreased to 120.41 Mb (74.3% reduction)
Mem. usage decreased to 114.92 Mb (75.0% reduction)
Mem. usage decreased to 111.43 Mb (75.7% reduction)
Mem. usage decreased to 85.79 Mb (74.3% reduction)


In [ ]:
# pk.dump(assets,open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/base_mod.pkl','wb'))

In [7]:
assets=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/base_mod.pkl','rb'))

In [8]:
# del base_treino

In [9]:
assets[0].sample(100)

,Count,Open,High,Low,Close,Volume,VWAP,Target,dia_semana,hora,...,boll_30,Didi,MACD,willr,willr_flag,log_ret,upper_shadow_perc,lower_shadow_perc,upper_shadow_15_perc,lower_shadow_15_perc
timestamp,,,,,,,,,,,,,,,,,,,,,
1618534740,1981.0,533.500000,536.000000,533.500000,534.500000,6837.198730,535.000000,-0.007721,4,0,...,0.0,0.0,-1.0,-59.312500,0.0,0.001474,0.002623,0.000586,0.000552,0.011200
1578592800,11.0,14.250000,14.250000,14.242188,14.242188,19.670000,14.242188,0.001918,3,18,...,0.0,0.0,-1.0,-86.750000,-1.0,-0.000576,0.000000,0.000000,-0.001433,0.003218
1615384020,688.0,285.250000,285.500000,285.000000,285.250000,2283.016357,285.250000,-0.002428,2,13,...,0.0,0.0,-1.0,-79.000000,0.0,0.000380,0.000775,0.000945,0.006927,0.004017
1546576740,22.0,5.894531,5.894531,5.886719,5.886719,3342.060059,5.890625,0.000731,4,4,...,-1.0,0.0,-1.0,-99.750000,-1.0,-0.001290,0.000000,0.000017,0.004147,0.002583
1537615920,1.0,10.109375,10.109375,10.109375,10.109375,1.500000,10.109375,-0.002670,5,11,...,0.0,0.0,1.0,-3.507812,1.0,0.001158,0.000000,0.000000,0.000079,0.003843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559429040,17.0,32.937500,32.937500,32.875000,32.906250,480.140015,32.906250,-0.002199,5,22,...,0.0,1.0,1.0,-24.062500,0.0,-0.001883,0.000206,0.000632,0.008896,-0.000083
1622000280,833.0,359.000000,360.000000,358.750000,359.750000,2371.370850,359.250000,0.002752,2,3,...,0.0,0.0,-1.0,-4.019531,1.0,0.002039,0.000186,0.000711,0.004570,0.003500
1583623500,35.0,20.046875,20.046875,20.031250,20.031250,489.170013,20.031250,0.000827,5,23,...,0.0,0.0,-1.0,-67.437500,0.0,-0.000723,0.000000,0.000145,0.000253,0.005241


In [10]:
assets[1].isnull().sum()

Count                   0
Open                    0
High                    0
Low                     0
Close                   0
Volume                  0
VWAP                    0
Target                  0
dia_semana              0
hora                    0
SMA_7                   0
SMA_15                  0
SMA_30                  0
SMA_60                  0
SMA_180                 0
log_SMA_7               0
log_SMA_14              0
log_SMA_30              0
log_SMA_60              0
log_SMA_180             0
log_CMA_30              0
log_CMA_15              0
log_EWM_30              0
log_EWM_15              0
boll_30                 0
Didi                    0
MACD                    0
willr                   0
willr_flag              0
log_ret                 0
upper_shadow_perc       0
lower_shadow_perc       0
upper_shadow_15_perc    0
lower_shadow_15_perc    0
dtype: int64

## 3.2 Features específicas (opcionais)

### 3.2.1 BTC

### 3.2.1 ETH

# 4. Modelagem

## 4.1 Quebra de treino/validação/teste

In [11]:
# list_periodos = []

# for asset in base_treino.Asset_ID.unique():
#     maximo = base_treino[base_treino.Asset_ID == asset].timestamp.max()
#     minimo = base_treino[base_treino.Asset_ID == asset].timestamp.min()
#     list_periodos.append([asset,minimo,maximo])
    
# df_periodos = pd.DataFrame(list_periodos, columns=['asset','min','max'])
# df_periodos['min'] = pd.to_datetime(df_periodos['min'], unit='s')
# df_periodos['max'] = pd.to_datetime(df_periodos['max'], unit='s')
# df_periodos.sort_values('min')

In [12]:
# y_treino={x: np.array(assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")]["Target"]) for x in assets.keys()}

In [13]:
# y_teste={x: np.array(assets[x].loc[totimestamp("01/04/2021"):totimestamp("01/07/2021")]["Target"]) for x in assets.keys()}

In [14]:
# x_treino = {x: assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1) for x in assets.keys()}

In [15]:
# x_teste = {x: assets[x].loc[totimestamp("01/04/2021"):totimestamp("01/07/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1) for x in assets.keys()}

In [16]:
from sklearn.preprocessing import StandardScaler
# simple preprocessing of the data 
scaler = StandardScaler()

In [17]:
x_treino = {x: scaler.fit_transform(assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1)) for x in assets.keys()}

x_teste = {x: scaler.fit_transform(assets[x].loc[totimestamp("02/04/2021"):totimestamp("01/07/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1)) for x in assets.keys()}

x_valid = {x: scaler.fit_transform(assets[x].loc[totimestamp("02/07/2021"):totimestamp("01/09/2021")].drop(['Open', 'High', 'Low', 'Close', 'VWAP','Target'], axis=1)) for x in assets.keys()}




In [18]:
y_treino={x: (np.array(assets[x].loc[totimestamp("01/01/2020"):totimestamp("01/04/2021")]["Target"])) for x in assets.keys()}

y_teste={x: (np.array(assets[x].loc[totimestamp("02/04/2021"):totimestamp("01/07/2021")]["Target"])) for x in assets.keys()}

y_valid={x: np.array(assets[x].loc[totimestamp("02/07/2021"):totimestamp("01/09/2021")]["Target"]) for x in assets.keys()}


4.12 Bases para modelo multioutput

In [40]:
x_all_treino = np.concatenate((x_treino[0], x_treino[1],x_treino[2],x_treino[3],x_treino[4],x_treino[5],x_treino[6],x_treino[7],x_treino[8],x_treino[9],x_treino[10],x_treino[11],x_treino[12],x_treino[13]), axis=1)

x_all_teste = np.concatenate((x_teste[0], x_teste[1],x_teste[2],x_teste[3],x_teste[4],x_teste[5],x_teste[6],x_teste[7],x_teste[8],x_teste[9],x_teste[10],x_teste[11],x_teste[12],x_teste[13]), axis=1)

x_all_valid = np.concatenate((x_valid[0], x_valid[1],x_valid[2],x_valid[3],x_valid[4],x_valid[5],x_valid[6],x_valid[7],x_valid[8],x_valid[9],x_valid[10],x_valid[11],x_valid[12],x_valid[13]), axis=1)


In [41]:
y_all_treino = np.column_stack((y_treino[0], y_treino[1],y_treino[2],y_treino[3],y_treino[4],y_treino[5],y_treino[6],y_treino[7],y_treino[8],y_treino[9],y_treino[10],y_treino[11],y_treino[12],y_treino[13]))

y_all_teste = np.column_stack((y_teste[0], y_teste[1],y_teste[2],y_teste[3],y_teste[4],y_teste[5],y_teste[6],y_teste[7],y_teste[8],y_teste[9],y_teste[10],y_teste[11],y_teste[12],y_teste[13]))

y_all_valid = np.column_stack((y_valid[0], y_valid[1],y_valid[2],y_valid[3],y_valid[4],y_valid[5],y_valid[6],y_valid[7],y_valid[8],y_valid[9],y_valid[10],y_valid[11],y_valid[12],y_valid[13]))

## 4.2 Modelo baseline

In [ ]:
modelos_reg_lin ={}
import pickle as pk
from sklearn import linear_model
for base in assets:
  modelos_reg_lin[base] = linear_model.Ridge(alpha=.1)
  modelos_reg_lin[base].fit(x_treino[base], y_treino[base])
  pk.dump(modelos_reg_lin[base],open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_lin_reg/modelo_lin_reg'+str(base)+'.pkl','wb'))

In [ ]:
del  modelos_reg_lin

## 4.3 Modelagem freestyle

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

mlr = MultiOutputRegressor(linear_model.Ridge(alpha=.1))
mlr.fit(x_all_treino,y_all_treino)

KeyboardInterrupt: ignored

In [ ]:
pk.dump(mlr,open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_multi_output.pkl','wb'))

4.4 MODELAGEM MARCELO

4.41 MODELO XGB COM PARAMETER SAMPLER

In [ ]:
# from sklearn.model_selection import ParameterSampler
# param_dist = {
#     "n_estimators": [100],
#     "max_depth": [4,6],
#     "colsample_bytree":[0.9],
#     "min_samples_split": [5],
#     "reg_lambda":[7],
#     "reg_alpha": [6],
#     "eta":[0.05],
#     "scale_pos_weight": [15],
#     "learning_rate": [0.2],
#     "gamma":[0.1],
#     "n_jobs":[-1]
#     # "missing":[0]

# }
# params_set=ParameterSampler(param_dist,2,random_state=0)

In [ ]:
from sklearn.model_selection import ParameterSampler
param_dist = {
    "n_estimators": [100],
    "max_depth": [4,6,11],
    "colsample_bytree":[0.75],
    "subsample": [0.9],
    "learning_rate": [0.06],
    "n_jobs":[-1]
    # "missing":[0]

}
params_set=ParameterSampler(param_dist,3,random_state=0)

In [ ]:
# #Função objetivo customizada
# from typing import Tuple
# import xgboost as xgb
# def rmsle_xgb(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
#     ''' Root mean squared log error metric.'''
#     y = dtrain.get_label()
#     predt[predt < -1] = -1 + 1e-6
#     elements = np.power(np.log1p(y) - np.log1p(predt), 2)
#     return 'PyRMSLE', float(np.sqrt(np.sum(elements) / len(y)))

In [ ]:
def arruma_n_estimators(row):
  best_iter=row["best_iter"]
  params_inicial = row["params_iter"]
  params_com_early = {**params_inicial, **{'n_estimators':best_iter +1}}
  return params_com_early

In [ ]:
from tqdm import tqdm_notebook
import tqdm

In [ ]:
# https://xgboost.readthedocs.io/en/stable/tutorials/custom_metric_obj.html

In [ ]:
def roda_grid(params,x_train,y_train,eval_set,eval_metric,early_stopping_rounds=5,metrica="rmse"):
  for k,i in tqdm.tqdm(enumerate(params,1)):
    regressor=xgb.XGBRegressor (objective="reg:squarederror", seed=0,**i)

    regressor.fit(x_train,y_train,eval_set=eval_set,eval_metric=eval_metric,verbose=False,early_stopping_rounds=early_stopping_rounds)

    if k==1:
      df_result=pd.DataFrame()
      df_result["best_treino"]=[regressor.evals_result()["validation_0"][metrica][regressor.best_iteration]]
      df_result["best_test"]=[regressor.evals_result()["validation_1"][metrica][regressor.best_iteration]]
      df_result["best_iter"]=regressor.best_iteration
      df_result["Combination"]=k
      df_result["params_iter"]=[i.copy()]
    else:
      aux=pd.DataFrame()
      aux["best_treino"]=[regressor.evals_result()["validation_0"][metrica][regressor.best_iteration]]
      aux["best_test"]=[regressor.evals_result()["validation_1"][metrica][regressor.best_iteration]]
      aux["best_iter"]=regressor.best_iteration
      aux["Combination"]=k
      aux["params_iter"]=[i.copy()]


      df_result=df_result.append(aux).reset_index(drop=True)
      df_result["params_com_early"]=df_result.apply(arruma_n_estimators, axis=1)
      df_result["diff"]=df_result["best_treino"] - df_result["best_test"]
      df_result=(df_result.sort_values(by=["best_test","diff", "best_iter"],ascending=[True,True,True]).reset_index(drop=True))

  return df_result

In [ ]:
df_result={}
best_params={}
eval_set={}
# for base in assets:
base=0
eval_set[base] = [(x_treino[base],y_treino[base]),(x_teste[base],y_teste[base])]
print(base)
df_result[base]=roda_grid(params_set, x_treino[base], y_treino[base],eval_set[base],eval_metric="rmse",metrica="rmse")
best_params[base]=df_result[base].loc[0,"params_com_early"]


0


2it [04:22, 131.19s/it]


In [ ]:
modelos_xgb ={}
import pickle as pk
for base in assets:
  modelos_xgb[base] = xgb.XGBRegressor(objective="reg:squarederror",**best_params[base])
  modelos_xgb[base].fit(x_treino[base], y_treino[base],eval_metric="rmse",eval_set=eval_set[base])
  pk.dump(modelos_xgb[base],open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_xgb/modelo_xgb'+str(base)+'.pkl','wb'))

In [ ]:
del modelos_xgb

In [ ]:
modelos_xgb ={}
base=0
modelos_xgb[base] = xgb.XGBRegressor(objective="reg:squarederror",**best_params[base])
modelos_xgb[base].fit(x_treino[base], y_treino[base],eval_metric="rmse",eval_set=eval_set[base])
predito=modelos_xgb[base].predict(x_teste[base])
predito

[0]	validation_0-rmse:0.469925	validation_1-rmse:0.470009
[1]	validation_0-rmse:0.441785	validation_1-rmse:0.441813
[2]	validation_0-rmse:0.41522	validation_1-rmse:0.41531
[3]	validation_0-rmse:0.390352	validation_1-rmse:0.390397
[4]	validation_0-rmse:0.366933	validation_1-rmse:0.36698
[5]	validation_0-rmse:0.344924	validation_1-rmse:0.344966
[6]	validation_0-rmse:0.324249	validation_1-rmse:0.324276
[7]	validation_0-rmse:0.304788	validation_1-rmse:0.304826
[8]	validation_0-rmse:0.286496	validation_1-rmse:0.286542
[9]	validation_0-rmse:0.269301	validation_1-rmse:0.269357
[10]	validation_0-rmse:0.253166	validation_1-rmse:0.253204
[11]	validation_0-rmse:0.237973	validation_1-rmse:0.238019
[12]	validation_0-rmse:0.223696	validation_1-rmse:0.223747
[13]	validation_0-rmse:0.210278	validation_1-rmse:0.21033
[14]	validation_0-rmse:0.197668	validation_1-rmse:0.19772
[15]	validation_0-rmse:0.185812	validation_1-rmse:0.185866
[16]	validation_0-rmse:0.17467	validation_1-rmse:0.174725
[17]	validati

array([0.01231262, 0.01266712, 0.01353884, ..., 0.01179057, 0.01247266,
       0.01247266], dtype=float32)

In [ ]:
print('Test score for XGB:'+ str(base), f"{np.corrcoef(predito, y_teste[base])[0,1]:.2f}")

Test score for XGB:0 0.02


In [ ]:
predito=modelos_xgb[base].predict(x_valid[base])
predito

array([0.01300594, 0.01480502, 0.01480502, ..., 0.00330222, 0.00330222,
       0.00330222], dtype=float32)

In [ ]:
print('Test score for XGB:'+ str(base), f"{np.corrcoef(predito, y_valid[base])[0,1]:.2f}")

Test score for XGB:0 0.00


4.42 MODELO LGBM COM PARAMETER SAMPLER

In [50]:
from sklearn.model_selection import ParameterSampler
param_dist_LGBM = {
    "learning_rate": [0.06,0.2,0.5],
    "n_estimators": [150],
    "max_depth": [4,6,10],
    "colsample_bytree":[0.9],
    # "min_samples_split": [2,4,5],
    "reg_lambda":[1,7],
    "reg_alpha": [0,6],
    # "scale_pos_weight": [15],
    "n_jobs":[-1]

}
params_set_LGBM=ParameterSampler(param_dist_LGBM,40,random_state=0)

In [51]:
#Função objetivo customizada
from lightgbm import LGBMRegressor
def rmsle_LGBM(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

In [52]:
#Função personalizada que avalia os resultados

def feval_func(preds, train_data):
   # Define a formula that evaluates the results
    return ('feval_func_name', eval_result, False)

In [53]:
def arruma_n_estimators(row):
  best_iter=row["best_iter"]
  params_inicial = row["params_iter"]
  params_com_early = {**params_inicial, **{'n_estimators':best_iter +1}}
  return params_com_early

In [54]:
from tqdm import tqdm_notebook
import tqdm

In [55]:
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564

In [56]:
def roda_grid_LGBM(params,x_train,y_train,eval_set,eval_metric,early_stopping_rounds=5,metrica="rmse"):
  for k,i in tqdm.tqdm(enumerate(params,1)):
    regressor=LGBMRegressor (objective='regression', seed=0,**i)

    regressor.fit(x_train,y_train,eval_set=eval_set,eval_metric=eval_metric,verbose=False,early_stopping_rounds=early_stopping_rounds)

    if k==1:
      df_result=pd.DataFrame()
      df_result["best_treino"]=[regressor.evals_result_["training"][metrica][regressor.best_iteration_]]
      df_result["best_test"]=[regressor.evals_result_["valid_1"][metrica][regressor.best_iteration_]]
      df_result["best_iter"]=regressor.best_iteration_
      df_result["Combination"]=k
      df_result["params_iter"]=[i.copy()]
    else:
      aux=pd.DataFrame()
      aux["best_treino"]=[regressor.evals_result_["training"][metrica][regressor.best_iteration_]]
      aux["best_test"]=[regressor.evals_result_["valid_1"][metrica][regressor.best_iteration_]]
      aux["best_iter"]=regressor.best_iteration_
      aux["Combination"]=k
      aux["params_iter"]=[i.copy()]


      df_result=df_result.append(aux).reset_index(drop=True)
      df_result["params_com_early"]=df_result.apply(arruma_n_estimators, axis=1)
      df_result["diff"]=df_result["best_treino"] - df_result["best_test"]
      df_result=(df_result.sort_values(by=["best_test","diff", "best_iter"],ascending=[True,True,True]).reset_index(drop=True))

  return df_result

In [ ]:
df_result_LGBM={}
best_params_LGBM={}
eval_set_LGBM={}
for base in assets:
  eval_set_LGBM[base] = [(x_treino[base],y_treino[base]),(x_teste[base],y_teste[base])]
  print(base)
  df_result_LGBM[base]=roda_grid_LGBM(params_set_LGBM, x_treino[base], y_treino[base],eval_set=eval_set_LGBM[base],eval_metric="rmse")
  # df_result_LGBM[base]=roda_grid_LGBM(params_set_LGBM, x_treino[base], y_treino[base],eval_set=eval_set_LGBM[base],eval_metric=rmsle_LGBM)
  best_params_LGBM[base]=df_result_LGBM[base].loc[0,"params_com_early"]


2


36it [01:32,  2.57s/it]


0


34it [01:07,  1.82s/it]

In [ ]:
modelos_LGBM ={}
import pickle as pk
for base in assets:
  modelos_LGBM[base] = LGBMRegressor(objective="regression",**best_params_LGBM[base])
  modelos_LGBM[base].fit(x_treino[base], y_treino[base],eval_metric=rmsle_LGBM,eval_set=eval_set_LGBM[base],verbose=False)
  pk.dump(modelos_LGBM[base],open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_lgbm'+str(base)+'.pkl','wb'))

# 5. Avaliação dos resultados

5.1 Avaliação MODELO

In [48]:
predito_test_reg_lin={}
predito_test_xgb={}
predito_test_lgbm={}

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

mlr=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_multi_out/modelo_multi_output.pkl','rb'))
y_pred_all_teste = mlr.predict(x_all_teste)

for base in assets:

  print('Teste Moeda:' + str(base))

  modelos_reg_lin=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_lin_reg/modelo_lin_reg'+str(base)+'.pkl','rb'))
  predito_test_reg_lin[base]=modelos_reg_lin.predict(x_teste[base])
  print('Test score for LR baseline:'+ str(base), f"{np.corrcoef(predito_test_reg_lin[base], y_teste[base])[0,1]:.2f}")



  modelos_xgb=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_xgb/modelo_xgb'+str(base)+'.pkl','rb'))
  predito_test_xgb[base]=modelos_xgb.predict(x_teste[base])
  print('Test score for XGB:'+ str(base), f"{np.corrcoef(predito_test_xgb[base], y_teste[base])[0,1]:.2f}")



  modelos_lgbm=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_lgbm/modelo_lgbm'+str(base)+'.pkl','rb'))
  predito_test_lgbm[base]=modelos_lgbm.predict(x_teste[base])
  print('Teste score for LGBM:'+ str(base), f"{np.corrcoef(predito_test_lgbm[base], y_teste[base])[0,1]:.2f}")


  print('Teste score for MULTIOUTPUT:'+ str(base), f"{np.corrcoef(y_pred_all_teste[base], y_all_teste[base])[0,1]:.2f}")

  print("FIM")


Teste Moeda:2
Test score for LR baseline:2 0.07
Test score for XGB:2 0.03
Teste score for LGBM:2 0.03
Teste score for MULTIOUTPUT:2 0.01
FIM
Teste Moeda:0
Test score for LR baseline:0 -0.02
Test score for XGB:0 0.02
Teste score for LGBM:0 0.01
Teste score for MULTIOUTPUT:0 0.00
FIM
Teste Moeda:1
Test score for LR baseline:1 0.02
Test score for XGB:1 -0.01
Teste score for LGBM:1 0.01
Teste score for MULTIOUTPUT:1 0.03
FIM
Teste Moeda:5
Test score for LR baseline:5 0.04
Test score for XGB:5 0.01
Teste score for LGBM:5 0.02
Teste score for MULTIOUTPUT:5 0.11
FIM
Teste Moeda:7
Test score for LR baseline:7 0.01
Test score for XGB:7 -0.00
Teste score for LGBM:7 -0.01
Teste score for MULTIOUTPUT:7 0.17
FIM
Teste Moeda:6
Test score for LR baseline:6 0.03
Test score for XGB:6 -0.01
Teste score for LGBM:6 0.03
Teste score for MULTIOUTPUT:6 0.15
FIM
Teste Moeda:9
Test score for LR baseline:9 0.04
Test score for XGB:9 0.00
Teste score for LGBM:9 0.05
Teste score for MULTIOUTPUT:9 0.05
FIM
Teste Mo

In [49]:
predito_valid_reg_lin={}
predito_valid_xgb={}
predito_valid_lgbm={}

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

mlr=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_multi_out/modelo_multi_output.pkl','rb'))
y_pred_all_valid= mlr.predict(x_all_valid)


for base in assets:
  print('Validação Moeda:' + str(base))

  modelos_reg_lin=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_lin_reg/modelo_lin_reg'+str(base)+'.pkl','rb'))
  # modelo=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_'+str(base)+'.pkl','rb'))
  predito_valid_reg_lin[base]=modelos_reg_lin.predict(x_valid[base])
  print('Valid score for LR baseline:'+ str(base), f"{np.corrcoef(predito_valid_reg_lin[base], y_valid[base])[0,1]:.2f}")



  modelos_xgb=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_xgb/modelo_xgb'+str(base)+'.pkl','rb'))
  # modelo=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_'+str(base)+'.pkl','rb'))
  predito_valid_xgb[base]=modelos_xgb.predict(x_valid[base])
  print('Valid score for XGB:'+ str(base), f"{np.corrcoef(predito_valid_xgb[base], y_valid[base])[0,1]:.2f}")



  modelos_lgbm=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/mod_lgbm/modelo_lgbm'+str(base)+'.pkl','rb'))
  # modelo=pk.load(open('gdrive/My Drive/Colab Notebooks/Kaggle/g_research_competition/modelo_'+str(base)+'.pkl','rb'))
  predito_valid_lgbm[base]=modelos_lgbm.predict(x_valid[base])
  print('Valid score for LGBM:'+ str(base), f"{np.corrcoef(predito_valid_lgbm[base], y_valid[base])[0,1]:.2f}")


  
  print('Teste score for MULTIOUTPUT:'+ str(base), f"{np.corrcoef(y_pred_all_valid[base], y_all_valid[base])[0,1]:.2f}")

  print('FIM')
  


Validação Moeda:2
Valid score for LR baseline:2 0.00
Valid score for XGB:2 0.00
Valid score for LGBM:2 0.03
Teste score for MULTIOUTPUT:2 -0.36
FIM
Validação Moeda:0
Valid score for LR baseline:0 -0.02
Valid score for XGB:0 0.01
Valid score for LGBM:0 0.03
Teste score for MULTIOUTPUT:0 -0.42
FIM
Validação Moeda:1
Valid score for LR baseline:1 0.06
Valid score for XGB:1 -0.01
Valid score for LGBM:1 0.03
Teste score for MULTIOUTPUT:1 -0.37
FIM
Validação Moeda:5
Valid score for LR baseline:5 0.02
Valid score for XGB:5 0.01
Valid score for LGBM:5 0.03
Teste score for MULTIOUTPUT:5 -0.32
FIM
Validação Moeda:7
Valid score for LR baseline:7 0.02
Valid score for XGB:7 0.01
Valid score for LGBM:7 0.00
Teste score for MULTIOUTPUT:7 -0.09
FIM
Validação Moeda:6
Valid score for LR baseline:6 0.01
Valid score for XGB:6 -0.01
Valid score for LGBM:6 -0.00
Teste score for MULTIOUTPUT:6 -0.29
FIM
Validação Moeda:9
Valid score for LR baseline:9 0.04
Valid score for XGB:9 -0.00
Valid score for LGBM:9 0.05